In [21]:
!pip install -qU langchain-google-genai langchain langchain_community wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.7 MB/s eta 0:00:00


In [ ]:
# langchain
# langchain_openai
# langchain_community
# wikipedia
# langchainhub
# pillow
# jupyter

In [5]:
import getpass
import os

# You might have to do this on your phone!
# Create an API key at https://ai.google.dev/gemini-api/docs/api-key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [8]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-6273c53d-a1e0-44f1-aa6f-a15c7a8c1116-0', usage_metadata={'input_tokens': 20, 'output_tokens': 7, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})

In [12]:
print(llm.invoke("What are some popular flake8 plugins?").content)

Flake8 is a fantastic tool for enforcing code style and detecting errors in Python code. Its extensibility through plugins is one of its greatest strengths. Here are some popular and highly recommended Flake8 plugins, categorized for clarity:

**General Code Quality & Style:**

*   **flake8-bugbear:** (B901-B950)  This is a *must-have*.  It catches a wide range of common bugs and anti-patterns that are often missed by other linters.  It's very opinionated and often suggests improvements that lead to more robust and readable code.  Examples include:
    *   Unnecessary `else` after `return`
    *   Using `isinstance` with a tuple of types (suggests using `isinstance(x, (type1, type2))`)
    *   Using `len(x) == 0` instead of `not x`
    *   Using `try...except...pass` (which is almost always a bad idea)
*   **flake8-comprehensions:** (C400-C419)  Suggests using list/set/dict comprehensions instead of loops or `map`/`filter` when appropriate.  This can significantly improve code readabil

In [15]:
import sys
import os

import requests
# from helper_functions.keys import WEATHER_KEY, HUGGING_FACE_KEY
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
import io

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


---

#### Exercise 1.1 (a): Explore tool parameters <a id='1a'></a>

**Task:**  
Use the methods ``name``, ``description``, ``args``, ``return_direct``, ``metadata`` to familiarize yourself with the parameters of the tool. What is the meaning of the different parameters?

**Background:**
Each tool is a ``BaseTool`` class object, you can find its definition [here](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool).

In [16]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

**SOLUTION:**

In [17]:
print("Name: ", wiki_tool.name)
print("Description: ", wiki_tool.description)
print("Input argument schema: ", wiki_tool.args)
print("Return output to user? ", wiki_tool.return_direct)
print("Metadata: ", wiki_tool.metadata)

Name:  wikipedia
Description:  A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Input argument schema:  {'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}
Return output to user?  False
Metadata:  None


---

#### Exercise 1.1 (b): Run tool and explore output <a id='1b'></a>

**TASK:**
* Use the ``.run(tool_input)`` method to execute the tool. The ``tool_input`` is the search term that you'd like to query wikipedia with.
* [Optional] Check out the arguments of the WikipediaAPIWrapper [here](https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.wikipedia.WikipediaAPIWrapper.html) and modify its parameters above. How does the output change?

**SOLUTION:**

In [18]:
tool_input = 'Amsterdam'

print(wiki_tool.run(tool_input))

Page: Amsterdam
Summary: Amsterdam ( AM-stər-dam, UK also  AM-stər-DAM; Dutch: [ˌɑmstərˈdɑm] ; lit. 'Dam in the Amstel') is the capital and most populated city of the Netherlands. It has a population of 933,680 in June 2024 within the city proper, 1,457,018 in the urban area and 2,480,394 in the metropolitan area. Located in the Dutch province of North Holland, Amsterdam is colloquially referred to as the "Venice of the North", for its large number of canals, now a UNESCO World Heritage Site.
Amsterdam was founded at the mouth of the Amstel River, which was dammed to control flooding. Originally a small fishing village in the 12th century, Amsterdam became a major world port during the Dutch Golden Age of the 17th century, when the Netherlands was an economic powerhouse. Amsterdam was the leading centre for finance and trade, as well as a hub of secular art production. In the 19th and 20th centuries, the city expanded and new neighborhoods and suburbs were built. The city has a long tr

---

#### Exercise 1.2 (a): Build your own Weather tool <a id='2a'></a>
The goal is to build a tool that extracts weather information from the weather site visualcrossing.com. You typically need an API key to extract information from a website. In this example we provide you with the API key.

**TASK:**
- Build the tool by defining the input parameters and the descriptions. The tool function is already provided to you.
- Turn function, description and input parameters into a tool through ``StructuredTool.from_function()``.
- Test if the tool gives an output.

**SOLUTION:**

---

**SOLUTION:**

---

## 🤖 Part 2 Solutions: Agents


**Content:**

1. [Exercise 2.1: Explore the agent's output](#1)
2. [Exercise 2.2 : Build your own agent](#2)
3. [Exercise 2.3: Invoke as many tools as you can](#3)
4. [Exercise 2.4 : Optimize the agent prompt](#4)

In [54]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
from langchain_community.utilities import WikipediaAPIWrapper
import io
import requests

WIKI_API_WRAPPER = WikipediaAPIWrapper(top_k_results=1)

# ------------------------------------------------------------------------------------
##### Wikipedia tool #####
def wikipedia_caller(query:str) ->str:
    """This function queries wikipedia through a search query."""
    return WIKI_API_WRAPPER.run(query)

# Input parameter definition
class QueryInput(BaseModel):
    query: str = Field(description="Input search query")

# the tool description
# wiki_tool_description: str = (
#         "A wrapper around Wikipedia. "
#         "Useful only for when you need to answer general questions about places. Not other entities."
#         "Input should be a search query."
#     )
wiki_tool_description: str = (
        "A wrapper around Wikipedia. "
        "Useful for when you need to answer general questions about "
        "people, places, companies, facts, historical events, or other subjects. "
        "Input should be a search query."
    )

# fuse the function, input parameters and description into a tool.
my_own_wiki_tool = StructuredTool.from_function(
    func=wikipedia_caller,
    name="wikipedia",
    description=wiki_tool_description,
    args_schema=QueryInput,
    return_direct=False,
)

In [44]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
os.environ["TAVILY_API_KEY"] = "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"
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

In [41]:
import sys
import os

from langchain.agents import create_tool_calling_agent # set up the agent
from langchain.agents import AgentExecutor # execute agent
# from langchain_openai import ChatOpenAI # call openAI as agent llm
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI


---

#### Exercise 2.1: Explore the agent's output <a id='1'></a>

**TASK:**
In the examples below, read the output to observe how the Agent reasons and decides to use a different tool based on the context.

In [60]:
# Agent

# Load Tools
tools = [my_own_wiki_tool, tavily_tool]

# Load LLM
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=OPENAI_KEY)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# Get the prompt to use - you can modify this! With this you let the agent know what its purpose is.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a nice assistant. You only use Tivaly unless absolutely necessary."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Define  the agent (load the LLM and the list of tools)
agent = create_tool_calling_agent(llm = llm, tools = tools, prompt = prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Your agent is ready.")


Your agent is ready.


In [65]:
ask_gemini("How can I prioritise one tool over another when building an llm agent with lanchain?")

You can prioritize one tool over another in a Langchain LLM agent in several ways, depending on the specific scenario and the agent type you're using. Here's a breakdown of common strategies and considerations:

**1. Tool Description and Prompt Engineering (Most Common and Recommended)**

*   **Detailed Tool Descriptions:**  The most effective way to influence an agent's tool selection is through clear, concise, and informative tool descriptions.  The LLM relies heavily on these descriptions to understand what each tool does and when to use it.

    *   **Highlight the Preferred Tool's Strengths:**  In the preferred tool's description, explicitly mention the scenarios where it excels.  Use keywords that the LLM is likely to pick up on when faced with a relevant user query.
    *   **Emphasize Limitations of the Alternative Tool:**  In the description of the less preferred tool, subtly mention its limitations or situations where it might not be the best choice.  This doesn't mean making

In [64]:
def ask_gemini(question: str):
    print(llm.invoke(question).content)

**SOLUTION:**

In [61]:
question_1 = "Where is Amsterdam?"

print(f"Question 1: {question_1}")
agent_executor.invoke({"input": question_1})

Question 1: Where is Amsterdam?


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Amsterdam location'}`


Page: Amsterdam
Summary: Amsterdam ( AM-stər-dam, UK also  AM-stər-DAM; Dutch: [ˌɑmstərˈdɑm] ; lit. 'Dam in the Amstel') is the capital and most populated city of the Netherlands. It has a population of 933,680 in June 2024 within the city proper, 1,457,018 in the urban area and 2,480,394 in the metropolitan area. Located in the Dutch province of North Holland, Amsterdam is colloquially referred to as the "Venice of the North", for its large number of canals, now a UNESCO World Heritage Site.
Amsterdam was founded at the mouth of the Amstel River, which was dammed to control flooding. Originally a small fishing village in the 12th century, Amsterdam became a major world port during the Dutch Golden Age of the 17th century, when the Netherlands was an economic powerhouse. Amsterdam was the leading centre for finance and trade, as well as a hub of secula

{'input': 'Where is Amsterdam?',
 'output': 'Amsterdam is the capital and most populated city of the Netherlands, located in the province of North Holland.'}

In [62]:
question_1 = "What did Elon Musk do during the CPAC conference in 2025?"

print(f"Question 1: {question_1}")
agent_executor.invoke({"input": question_1})

Question 1: What did Elon Musk do during the CPAC conference in 2025?


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Elon Musk CPAC conference 2025'}`


[{'title': 'Musk waves a chainsaw and charms conservatives at CPAC', 'url': 'https://apnews.com/article/musk-chainsaw-trump-doge-6568e9e0cfc42ad6cdcfd58a409eb312', 'content': "![Image 3: Elon Musk holds up a chainsaw he received from Argentina's President Javier Milei, right, as they arrive to speak at the Conservative Political Action Conference, CPAC, at the Gaylord National Resort & Convention Center, Thursday, Feb. 20, 2025, in Oxon Hill, Md. ![Image 6: Elon Musk, left, receives a chainsaw from Argentina's President Javier Milei, right, as they arrive speaks at the Conservative Political Action Conference, CPAC, at the Gaylord National Resort & Convention Center, Thursday, Feb. 20, 2025, in Oxon Hill, Md. ![Image 10: Elon Musk, left, receives a chainsaw from Argentina's President Ja

{'input': 'What did Elon Musk do during the CPAC conference in 2025?',
 'output': 'Based on the search results, here\'s what Elon Musk did during the CPAC conference in 2025:\n\n*   **Received a chainsaw from Argentina\'s President Javier Milei:** He was photographed receiving the chainsaw as they arrived to speak at the conference.\n*   **Spoke at the conference:** He gave an interview at CPAC.\n*   **Was called a "genius engineer" and a "hero of free speech":** Despite some past disagreements, figures like Steve Bannon and Nigel Farage praised Musk at the event.'}

In [52]:
question_1 = "ING huis verduurzamen"

print(f"Question 1: {question_1}")
agent_executor.invoke({"input": question_1})

Question 1: ING huis verduurzamen


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'ING verduurzamen huis'}`


[{'title': 'ING introduceert ING Upgrader voor duurzamer en ...', 'url': 'https://www.duurzaam-beleggen.nl/2024/12/02/ing-introduceert-ing-upgrader-voor-duurzamer-en-toekomstbestendiger-wonen/', 'content': 'ING Upgrader vereenvoudigt de verduurzaming van woningen voor de honderdduizenden huishoudens die de bank financiert. Folkert van der Ploeg,', 'score': 0.80405265}, {'title': 'ING introduceert Upgrader voor duurzamer wonen - InFinance', 'url': 'https://www.infinance.nl/artikel/ing-introduceert-upgrader-voor-duurzamer-wonen/', 'content': 'ING lanceert vanaf 9 december ING Upgrader, een alles-in-één dienstverlening voor hypotheekklanten die hun woning willen verduurzamen.', 'score': 0.8025714}, {'title': 'Hypotheek voor een duurzaam huis - ING', 'url': 'https://www.ing.nl/particulier/hypotheek/hypotheekvormen/duurzaam-huis-kopen'

{'input': 'ING huis verduurzamen',
 'output': "ING offers multiple options to help you make your home more sustainable, including the ING Upgrader service and the possibility of borrowing more money on your mortgage if you're making your home more energy-efficient. They also offer information on different ways to finance the sustainability upgrades to your home."}

In [59]:
question_1 = "How long after launch did Starship explode during Starship Flight test 8?"

print(f"Question 1: {question_1}")
agent_executor.invoke({"input": question_1})

Question 1: How long after launch did Starship explode during Starship Flight test 8?


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Starship Flight test 8 explosion time after launch'}`


[{'title': 'FAA investigating SpaceX Starship Flight 8 explosion that disrupted ...', 'url': 'https://www.space.com/space-exploration/missions/faa-investigating-spacex-starship-flight-8-explosion-that-disrupted-commercial-flights', 'content': "SpaceX's latest Starship to attempt to reach space exploded nine minutes after its launch this week, leaving a trail of debris in its wake.", 'score': 0.882276}, {'title': 'Starship Explosions Show SpaceX No Longer Defying Gravity', 'url': 'https://www.nytimes.com/2025/03/08/science/starship-spacex-explosion-elon-musk.html', 'content': 'The latest explosion occurred on Thursday during the eighth test flight of Starship, less than two months after the seventh test flight also', 'score': 0.74273956}, {'title': "Sp

{'input': 'How long after launch did Starship explode during Starship Flight test 8?',
 'output': 'Starship exploded nine minutes after its launch during Starship Flight Test 8.'}

In [ ]:
[{'title': 'Elon Musk Speaks at CPAC | C-SPAN.org', 'url': 'https://www.c-span.org/clip/public-affairs-event/elon-musk-speaks-at-cpac/5154261', 'content': 'Elon Musk speaks at the Conservative Political Action Conference (CPAC) about his role at the "Department of Government Efficiency" (DOGE)', 'score': 0.38588423}, {'title': 'WATCH: Elon Musk waves chainsaw on stage at CPAC - YouTube', 'url': 'https://www.youtube.com/watch?v=nkMVb0RNptA', 'content': 'Billionaire Elon Musk began his appearance at the Conservative Political Action Conference brandishing a chainsaw that was gifted to him by', 'score': 0.32527456}, {'title': 'Musk waves a chainsaw and charms conservatives at CPAC', 'url': 'https://apnews.com/article/musk-chainsaw-trump-doge-6568e9e0cfc42ad6cdcfd58a409eb312', 'content': 'Billionaire Elon Musk has created a buzz waving a chainsaw in the air at a conservative gathering outside Washington.', 'score': 0.3052006}, {'title': 'Elon Musk becomes a new power center at CPAC - NBC News', 'url': 'https://www.nbcnews.com/politics/doge/elon-musk-becomes-new-power-center-cpac-rcna193097', 'content': 'Elon Musk stole the show at CPAC, a surprise guest who wielded a chainsaw to symbolize his government cuts.', 'score': 0.28808862}, {'title': "At CPAC, Elon Musk's 'Dark MAGA' takeover is embraced | AP News", 'url': 'https://apnews.com/article/trump-musk-cpac-doge-maga-bannon-96ad2a6989c1d210c3ea83ccc11069cd', 'content': 'At an annual gathering of conservatives, the emerging influence of billionaire Elon Musk on President Donald Trump was on display.', 'score': 0.20639811}]Elon Musk spoke at the Conservative Political Action Conference (CPAC) and brandished a chainsaw. He talked about his role at the "Department of Government Efficiency" (DOGE).


In [ ]:
question_1 = "Where is Amsterdam?"

print(f"Question 1: {question_1}")
agent_executor.invoke({"input": question_1})

In [38]:
question_1 = "Can you tell me something about Michael Jackson?"

print(f"Question 1: {question_1}")
agent_executor.invoke({"input": question_1})

Question 1: Can you tell me something about Michael Jackson?


> Entering new AgentExecutor chain...
I am sorry, I cannot help you with that. I can only answer general questions about places.

> Finished chain.


{'input': 'Can you tell me something about Michael Jackson?',
 'output': 'I am sorry, I cannot help you with that. I can only answer general questions about places.'}

In [30]:
question_1 = "Can you tell me something about Michael Jackson?"

print(f"Question 1: {question_1}")
agent_executor.invoke({"input": question_1})

Question 1: Can you tell me something about Michael Jackson?


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Michael Jackson'}`


Page: Michael Jackson
Summary: Michael Joseph Jackson (August 29, 1958 – June 25, 2009) was an American singer, songwriter, dancer, and philanthropist. Dubbed the "King of Pop", he is regarded as one of the most significant figures of the 20th century. Over a four-decade career, his world record music achievements broke racial barriers in America and made him a global figure. Through songs, stages, and fashion, he proliferated visual performance for artists in popular music; popularizing street dance moves including the moonwalk (which he named), the robot, and the anti-gravity lean. Jackson is often deemed the greatest entertainer of all time based on his acclaim and records.
The eighth child of the Jackson family, Michael made his public debut in 1964 at age six, as a member of the Jackson 5 (later known as the Jacksons). Af

{'input': 'Can you tell me something about Michael Jackson?',
 'output': 'Michael Joseph Jackson (1958-2009) was an American singer, songwriter, dancer, and philanthropist, dubbed the "King of Pop." He is regarded as one of the most significant figures of the 20th century and broke racial barriers in America. Jackson made his public debut in 1964 at age six, as a member of the Jackson 5. He achieved solo stardom with the release of his fifth album Off the Wall (1979), and followed it up with Thriller (1982), the best-selling album of all time. Jackson is one of the best-selling music artists of all time, with estimated sales of over 500 million records worldwide. He had 13 Billboard Hot 100 number-one singles and is the first artist to have a top-ten single on the Billboard Hot 100 in five different decades. From the late 1980s, Jackson became a figure of controversy and speculation due to his changing appearance, relationships, behavior, and lifestyle. He was accused of sexually abusi

In [ ]:
question_2 = "Can you ?"

print(f"Question 2: {question_2}")
agent_executor.invoke({"input": question_2})

Question 2: What is the current temperature in Amsterdam?


> Entering new AgentExecutor chain...

Invoking: `weather` with `{'city': 'Amsterdam'}`


Current temperature in Amsterdam: 15.4°CThe current temperature in Amsterdam is 15.4°C.

> Finished chain.


{'input': 'What is the current temperature in Amsterdam?',
 'output': 'The current temperature in Amsterdam is 15.4°C.'}

In [ ]:
question_3 = "What should I visit in Amsterdam? Show me an photo"

print(f"Question 1: {question_3}")
agent_executor.invoke({"input": question_3})

Question 1: What should I visit in Amsterdam? Show me an photo


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Tourist attractions in Amsterdam'}`


Page: List of tourist attractions in Amsterdam
Summary: Amsterdam, one of Europe's capitals, has many attractions for visitors. The city's most famous sight is the 17th-century canals of Amsterdam (in Dutch: grachtengordel), located in the heart of Amsterdam, have been added to the UNESCO World Heritage List.
Invoking: `create_image` with `{'payload': 'Tourist attractions in Amsterdam'}`


images/image_Tourist_attractions_in_Amsterdam.jpg Amsterdam offers a variety of attractions for visitors. One of the most famous sights is the 17th-century canals, known as the "grachtengordel," which are located in the heart of the city and have been added to the UNESCO World Heritage List.

Here is an image of some tourist attractions in Amsterdam:

![Tourist attractions in Amsterdam](images/image_Tourist_attractions_in_

{'input': 'What should I visit in Amsterdam? Show me an photo',
 'output': 'Amsterdam offers a variety of attractions for visitors. One of the most famous sights is the 17th-century canals, known as the "grachtengordel," which are located in the heart of the city and have been added to the UNESCO World Heritage List.\n\nHere is an image of some tourist attractions in Amsterdam:\n\n![Tourist attractions in Amsterdam](images/image_Tourist_attractions_in_Amsterdam.jpg)'}

---

#### Exercise 2.2 : Build your own agent  <a id='2'></a>
The goal is to build an agent that uses the tools you previously developed. Feel free to also use the pre-made tools, available at `helper_functions/tools.py`

**TASK**:
- A template for defining an agent and an API key are already provided to you. Build an agent by using a list of tools, and the LLM `gpt-4o-mini`. A template for this agent follows below.
- Test if the agent gives an output.
- Observe how the output changes if you provide less tools in your list of tools.

**SOLUTION:**

In [ ]:
# Component 1 (Tools): Load Tools
tools = [
    my_own_wiki_tool, weather_tool, image_tool
]

# Component 2 (LLM): Load LLM
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0,
                 api_key=OPENAI_KEY)

# Component 3 (Prompt): Let the agent know what its purpose is. For now, let's keep it as is.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a nice assistant"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Define the agent and agent executor (load the LLM, the list of tools, and the prompt (descripiton))

agent = create_tool_calling_agent(
    llm = llm, tools = tools, prompt = prompt
    )
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

print("Your agent is ready.")


Your agent is ready.


---

#### Exercise 2.3: Invoke as many tools as you can  <a id='3'></a>

**TASK:**
Try various questions to call the agent and follow the generated reasoning process in the response. The goal is to call the agent in a way thay it will use as many tools as possible. **Let's see who can reach the highest number of tools used with a single prompt!**

**SOLUTION:** (invoking 2 tools: Wiki and Weather. Can you reach more?)

In [ ]:
question = """
Where is Amsterdam and what is the weather like?
"""

print(f"Question: {question}")
agent_executor.invoke({"input": question})

Question: 
Where is Amsterdam and what is the weather like?



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Amsterdam'}`


Page: Amsterdam
Summary: Amsterdam ( AM-stər-dam, UK also  AM-stər-DAM, Dutch: [ˌɑmstərˈdɑm] ; literally, "Dam in the Amstel") is the capital and most populated city of the Netherlands. It has a population of 921,402 within the city proper, 1,457,018 in the urban area and 2,480,394 in the metropolitan area. Located in the Dutch province of North Holland, Amsterdam is colloquially referred to as the "Venice of the North", for its large number of canals, now a UNESCO World Heritage Site.
Amsterdam was founded at the mouth of the Amstel River, which was dammed to control flooding. Originally a small fishing village in the 12th century, Amsterdam became a major world port during the Dutch Golden Age of the 17th century, when the Netherlands was an economic powerhouse. Amsterdam was the leading centre for finance and trade, as well as a 

{'input': '\nWhere is Amsterdam and what is the weather like?\n',
 'output': '### Location of Amsterdam:\nAmsterdam is the capital and most populated city of the Netherlands. It is located in the Dutch province of North Holland. The city is known for its extensive canal system, which has earned it the nickname "Venice of the North." Amsterdam was founded at the mouth of the Amstel River and has a rich history dating back to the 12th century. It became a major world port during the Dutch Golden Age in the 17th century and is now a cultural and financial hub.\n\n### Current Weather in Amsterdam:\nThe current temperature in Amsterdam is 15.4°C.'}

---

#### Exercise 2.4 : Optimize the agent prompt  <a id='4'></a>
So far we played around with the provided LLM and list of tools. Now let's look into the 3rd component: the Agent prompt.

**TASK:**
- Modify the agent prompt and observe the difference in the output.

**SOLUTION:**

In this solution we assume you are building an agent for a travel agency, and you want the agent to help you motivate people to visit a city.

In [ ]:
# Component 1 (Tools): Load Tools from Exercise 1
tools = tools

# Component 2 (LLM): Load LLM form Exercise 1
llm = llm

your_prompt = """"
    You are a helpful assisstant, trying to motivate people to go on holiday.
"""

# Component 3 (Prompt): Create your own prompt to instruct the Agent about its purpose.
prompt = ChatPromptTemplate.from_messages([
    ("system", your_prompt),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
prompt.messages


# Define the agent and agent executor (load the LLM, the list of tools, and the prompt (descripiton))
# This is same as in Exercise 1
agent = create_tool_calling_agent(
    llm = llm, tools = tools, prompt = prompt
    )
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

print("Your agent is ready.")


Your agent is ready.


In [ ]:
# Observe how the same question from before is answered differently with the different prompt.
print(f"Question: {question}")
agent_executor.invoke({"input": question})

Question: 
Where is Amsterdam and what is the weather like?



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Amsterdam'}`


Page: Amsterdam
Summary: Amsterdam ( AM-stər-dam, UK also  AM-stər-DAM, Dutch: [ˌɑmstərˈdɑm] ; literally, "Dam in the Amstel") is the capital and most populated city of the Netherlands. It has a population of 921,402 within the city proper, 1,457,018 in the urban area and 2,480,394 in the metropolitan area. Located in the Dutch province of North Holland, Amsterdam is colloquially referred to as the "Venice of the North", for its large number of canals, now a UNESCO World Heritage Site.
Amsterdam was founded at the mouth of the Amstel River, which was dammed to control flooding. Originally a small fishing village in the 12th century, Amsterdam became a major world port during the Dutch Golden Age of the 17th century, when the Netherlands was an economic powerhouse. Amsterdam was the leading centre for finance and trade, as well as a 

---